# Bring your own LLMs

Ragas uses langchain under the hood for connecting to LLMs for metrices that require them. This means you can swap out the default LLM we use (`gpt-3.5-turbo-16k`) to use any 100s of API supported out of the box with langchain.

- [Completion LLMs Supported](https://api.python.langchain.com/en/latest/api_reference.html#module-langchain.llms)
- [Chat based LLMs Supported](https://api.python.langchain.com/en/latest/api_reference.html#module-langchain.chat_models)

This guide will show you how to use another or LLM API for evaluation.

## Evaluating with GPT-3.5-turbo-instruct

Ragas uses gpt-3.5 by default. After failing the approach of "bring your own LLMs" for gpt-3.5-turbo-instruct, we directly change the codebase of v0.0.14 to use gpt-3.5-turbo-instruct for evaluation.
```
git diff src/ragas/metrics/base.py
diff --git a/src/ragas/metrics/base.py b/src/ragas/metrics/base.py
index 19868c3..5898f4c 100644
+from langchain.llms import OpenAI
 from langchain.chat_models.base import BaseChatModel
 from langchain.llms.base import BaseLLM
 from tqdm import tqdm
 
+
 if t.TYPE_CHECKING:
     from langchain.callbacks.base import Callbacks
 
@@ -107,7 +108,7 @@ class Metric(ABC):
 
 
 def _llm_factory():
-    return ChatOpenAI(model_name="gpt-3.5-turbo-16k")  # type: ignore
+    return OpenAI(model_name="gpt-3.5-turbo-instruct")  # type: ignore
 
 
 @dataclass
```

In [1]:
%pip show ragas

Name: ragas
Version: 0.0.15.dev0+ge194caa.d20230923
Summary: 
Home-page: 
Author: 
Author-email: 
License: 
Location: /home/inflaton/miniconda3/envs/ragas/lib/python3.11/site-packages
Editable project location: /home/inflaton/code/gpt/ragas_extended
Requires: datasets, langchain, numpy, openai, pydantic, pysbd, sentence-transformers, transformers
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

# os.environ["OPENAI_API_KEY"] 

True

In [3]:
# data
from datasets import load_dataset

fiqa_eval = load_dataset("explodinggradients/fiqa", "ragas_eval")
fiqa_eval

DatasetDict({
    baseline: Dataset({
        features: ['question', 'ground_truths', 'answer', 'contexts'],
        num_rows: 30
    })
})

In [4]:
dataset = fiqa_eval["baseline"]
new_ds = dataset.map(
    lambda record: {
        "answer": record["ground_truths"][0],
    },
    batched=False,
)
new_ds

Dataset({
    features: ['question', 'ground_truths', 'answer', 'contexts'],
    num_rows: 30
})

In [5]:
new_ds.to_pandas()

,question,ground_truths,answer,contexts
0,How to deposit a cheque issued to an associate...,[Have the check reissued to the proper payee.J...,Have the check reissued to the proper payee.Ju...,[Just have the associate sign the back and the...
1,Can I send a money order from USPS as a business?,[Sure you can. You can fill in whatever you w...,Sure you can. You can fill in whatever you wa...,[Sure you can. You can fill in whatever you w...
2,1 EIN doing business under multiple business n...,[You're confusing a lot of things here. Compan...,You're confusing a lot of things here. Company...,[You're confusing a lot of things here. Compan...
3,Applying for and receiving business credit,"[""I'm afraid the great myth of limited liabili...","""I'm afraid the great myth of limited liabilit...",[Set up a meeting with the bank that handles y...
4,401k Transfer After Business Closure,[You should probably consult an attorney. Howe...,You should probably consult an attorney. Howev...,[The time horizon for your 401K/IRA is essenti...
5,What are the ins/outs of writing equipment pur...,[Most items used in business have to be deprec...,Most items used in business have to be depreci...,[You would report it as business income on Sch...
6,Can a entrepreneur hire a self-employed busine...,[Yes. I can by all means start my own company ...,Yes. I can by all means start my own company a...,[Yes. I can by all means start my own company ...
7,Intentions of Deductible Amount for Small Busi...,"[""If your sole proprietorship losses exceed al...","""If your sole proprietorship losses exceed all...","[""Short answer, yes. But this is not done thro..."
8,How can I deposit a check made out to my busin...,[You should have a separate business account. ...,You should have a separate business account. M...,"[""I have checked with Bank of America, and the..."
9,Filing personal with 1099s versus business s-c...,[Depends whom the 1099 was issued to. If it wa...,Depends whom the 1099 was issued to. If it was...,[Depends whom the 1099 was issued to. If it wa...


In [6]:
%%time
# evaluate
from ragas import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
)

result = evaluate(
    new_ds,
    metrics=[
        faithfulness,
        answer_relevancy,
    ],
)

result

evaluating with [faithfulness]


100%|██████████| 2/2 [00:13<00:00,  6.81s/it]


evaluating with [answer_relevancy]


100%|██████████| 2/2 [00:43<00:00, 21.74s/it]


CPU times: user 1.95 s, sys: 457 ms, total: 2.4 s
Wall time: 1min


{'ragas_score': 0.8115, 'faithfulness': 0.7832, 'answer_relevancy': 0.8418}

In [7]:
df = result.to_pandas()
best_ds = df[(df.faithfulness > 0.99)]
best_ds

,question,contexts,answer,ground_truths,faithfulness,answer_relevancy
1,Can I send a money order from USPS as a business?,[Sure you can. You can fill in whatever you w...,Sure you can. You can fill in whatever you wa...,[Sure you can. You can fill in whatever you w...,1.0,0.832527
2,1 EIN doing business under multiple business n...,[You're confusing a lot of things here. Compan...,You're confusing a lot of things here. Company...,[You're confusing a lot of things here. Compan...,1.0,0.776302
3,Applying for and receiving business credit,[Set up a meeting with the bank that handles y...,"""I'm afraid the great myth of limited liabilit...","[""I'm afraid the great myth of limited liabili...",1.0,0.825436
7,Intentions of Deductible Amount for Small Busi...,"[""Short answer, yes. But this is not done thro...","""If your sole proprietorship losses exceed all...","[""If your sole proprietorship losses exceed al...",1.0,0.777188
9,Filing personal with 1099s versus business s-c...,[Depends whom the 1099 was issued to. If it wa...,Depends whom the 1099 was issued to. If it was...,[Depends whom the 1099 was issued to. If it wa...,1.0,0.850046
10,Using credit card points to pay for tax deduct...,"[""For simplicity, let's start by just consider...","""For simplicity, let's start by just consideri...","[""For simplicity, let's start by just consider...",1.0,0.865250
12,Investing/business with other people's money: ...,"[""Basically, you either borrow money, or get o...","""Basically, you either borrow money, or get ot...","[""Basically, you either borrow money, or get o...",1.0,0.875244
13,What approaches are there for pricing a small ...,"[I don't have any experience in this, but this...","I don't have any experience in this, but this ...","[I don't have any experience in this, but this...",1.0,0.851268
14,How to account for money earned and spent prio...,[Funds earned and spent before opening a dedic...,Funds earned and spent before opening a dedica...,[Funds earned and spent before opening a dedic...,1.0,0.912717
15,Do I need a new EIN since I am hiring employee...,[You don't need to notify the IRS of new membe...,I called the IRS (click here for IRS contact i...,[I called the IRS (click here for IRS contact ...,1.0,0.820586


In [8]:
best_ds.index

Index([1, 2, 3, 7, 9, 10, 12, 13, 14, 15, 16, 22, 24, 26, 27], dtype='int64')

In [9]:
baseline_index = [3, 9, 14, 16, 22, 24, 26, 27]
baseline_ds = new_ds.select(baseline_index)
baseline_ds.to_pandas()

,question,ground_truths,answer,contexts
0,Applying for and receiving business credit,"[""I'm afraid the great myth of limited liabili...","""I'm afraid the great myth of limited liabilit...",[Set up a meeting with the bank that handles y...
1,Filing personal with 1099s versus business s-c...,[Depends whom the 1099 was issued to. If it wa...,Depends whom the 1099 was issued to. If it was...,[Depends whom the 1099 was issued to. If it wa...
2,How to account for money earned and spent prio...,[Funds earned and spent before opening a dedic...,Funds earned and spent before opening a dedica...,[Funds earned and spent before opening a dedic...
3,Privacy preferences on creditworthiness data,[See the first item in the list: For our every...,See the first item in the list: For our everyd...,[See the first item in the list: For our every...
4,Challenged an apparently bogus credit card cha...,[Yes. For $15 it's not worth anyone's time to...,Yes. For $15 it's not worth anyone's time to ...,[If you tell the collector that the claim isn'...
5,Following an investment guru a good idea?,"[""The best answer here is """"maybe, but probabl...","""The best answer here is """"maybe, but probably...",[I think following the professional money mana...
6,Should a retail trader bother about reading SE...,"[""There are many different kinds of SEC filing...","""There are many different kinds of SEC filings...","[""There are many different kinds of SEC filing..."
7,Will one’s education loan application be rejec...,[A bank can reject a loan if they feel you do ...,A bank can reject a loan if they feel you do n...,[Edit: lazy math The answer to this question d...


In [12]:
%%time
# evaluate
from ragas import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
)

result_baseline = evaluate(
    baseline_ds,
    metrics=[
        faithfulness,
        answer_relevancy,
    ],
)

result_baseline

evaluating with [faithfulness]


100%|██████████| 1/1 [00:05<00:00,  5.11s/it]


evaluating with [answer_relevancy]


100%|██████████| 1/1 [00:09<00:00,  9.05s/it]


CPU times: user 100 ms, sys: 0 ns, total: 100 ms
Wall time: 15.2 s


{'ragas_score': 0.9180, 'faithfulness': 1.0000, 'answer_relevancy': 0.8484}

In [13]:
result_baseline.to_pandas()

,question,contexts,answer,ground_truths,faithfulness,answer_relevancy
0,Applying for and receiving business credit,[Set up a meeting with the bank that handles y...,"""I'm afraid the great myth of limited liabilit...","[""I'm afraid the great myth of limited liabili...",1.0,0.827099
1,Filing personal with 1099s versus business s-c...,[Depends whom the 1099 was issued to. If it wa...,Depends whom the 1099 was issued to. If it was...,[Depends whom the 1099 was issued to. If it wa...,1.0,0.847459
2,How to account for money earned and spent prio...,[Funds earned and spent before opening a dedic...,Funds earned and spent before opening a dedica...,[Funds earned and spent before opening a dedic...,1.0,0.909440
3,Privacy preferences on creditworthiness data,[See the first item in the list: For our every...,See the first item in the list: For our everyd...,[See the first item in the list: For our every...,1.0,0.813747
4,Challenged an apparently bogus credit card cha...,[If you tell the collector that the claim isn'...,Yes. For $15 it's not worth anyone's time to ...,[Yes. For $15 it's not worth anyone's time to...,1.0,0.874177
5,Following an investment guru a good idea?,[I think following the professional money mana...,"""The best answer here is """"maybe, but probably...","[""The best answer here is """"maybe, but probabl...",1.0,0.839353
6,Should a retail trader bother about reading SE...,"[""There are many different kinds of SEC filing...","""There are many different kinds of SEC filings...","[""There are many different kinds of SEC filing...",1.0,0.854352
7,Will one’s education loan application be rejec...,[Edit: lazy math The answer to this question d...,A bank can reject a loan if they feel you do n...,[A bank can reject a loan if they feel you do ...,1.0,0.821634
